Import all

In [ ]:
pip install nibabel numpy scipy matplotlib scikit-image torch torchvision pandas

GPU Available?

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

VGG19

In [ ]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import scipy.ndimage
from torch.cuda.amp import autocast, GradScaler

# ========== CONFIGURACIÓN ==========
CSV_PATH = r"D:/ADNI Data/BETandGroups.csv"
IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 2
EPOCHS = 50
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Optimización GPU
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

# ========== DATASET ==========
class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]

        volume = nib.load(path).get_fdata().astype(np.float32)
        zoom_factors = [t / c for t, c in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)

        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)

        volume = torch.tensor(volume).unsqueeze(0)
        label = torch.tensor(self.label_encoder.transform([label])[0])

        return volume, label

# ========== MODELO VGG19 3D ==========
class VGG19_3D(nn.Module):
    def __init__(self, num_classes):
        super(VGG19_3D, self).__init__()

        def conv_block(in_channels, out_channels, num_convs):
            layers = []
            for _ in range(num_convs):
                layers.append(nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1))
                layers.append(nn.ReLU(inplace=True))
                in_channels = out_channels
            layers.append(nn.MaxPool3d(kernel_size=2, stride=2))
            return nn.Sequential(*layers)

        self.features = nn.Sequential(
            conv_block(1, 64, 2),
            conv_block(64, 128, 2),
            conv_block(128, 256, 4),
            conv_block(256, 512, 4),
            conv_block(512, 512, 4),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 4 * 4 * 4, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# ========== CARGA DE DATOS ==========
# Asegúrate de tener tu DataFrame "data" cargado
# df = pd.read_csv("rutas.csv") o como sea que lo tengas

df = pd.read_csv(CSV_PATH)
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])

df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)

train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# ========== MODELO Y ENTRENAMIENTO ==========
model = VGG19_3D(num_classes=len(le.classes_)).to(DEVICE)
model.load_state_dict(torch.load(r"D:\ADNI Data\vgg19_3d_alzheimer_reentrenado.pth"))  # <-- IMPORTAR MODELO

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, min_lr=1e-8, verbose=True)
scaler = GradScaler()

for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0, 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for inputs, labels in loop:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)
    print(f"✅ Epoch {epoch+1} - Loss: {train_loss:.4f}, Train Acc: {acc_train:.2f}%")

    # Validación
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    acc_val = 100 * val_correct / len(val_dataset)
    print(f"🔍 Val Acc: {acc_val:.2f}%")

    torch.cuda.empty_cache()  # limpiar memoria en cada epoch

# ========== GUARDAR NUEVO MODELO ==========
torch.save(model.state_dict(), "vgg19_3d_alzheimer_reentrenado2.pth")
print("📦 Modelo reentrenado y guardado como vgg19_3d_alzheimer_reentrenado2.pth")


Entrenar mas epocas

In [ ]:
import torch
import torch.nn as nn
import nibabel as nib
import pandas as pd
import numpy as np
import scipy.ndimage
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

CSV_PATH = r"D:/ADNI Data/BETandGroups.csv"
IMG_SIZE = (128, 128, 128)
BATCH_SIZE = 1
EPOCHS = 50
LR = 1e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Nifti3DDataset(Dataset):
    def __init__(self, df, label_encoder):
        self.df = df.reset_index(drop=True)
        self.label_encoder = label_encoder
        self.target_shape = IMG_SIZE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.loc[idx, "Ruta"]
        label = self.df.loc[idx, "Grupo"]

        volume = nib.load(path).get_fdata().astype(np.float32)
        zoom_factors = [t / s for t, s in zip(self.target_shape, volume.shape)]
        volume = scipy.ndimage.zoom(volume, zoom=zoom_factors, order=3)

        volume = (volume - np.mean(volume)) / (np.std(volume) + 1e-8)
        volume = np.clip(volume, -3, 3)

        volume = torch.tensor(volume).unsqueeze(0) 
        label = torch.tensor(self.label_encoder.transform([label])[0])
        return volume, label

#Arquitectura VGG16
class VGG3D(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG3D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(64, 128, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(128, 128, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(128, 256, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(256, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2),

            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

#Cargar Datos
df = pd.read_csv(CSV_PATH)
le = LabelEncoder()
df['labels'] = le.fit_transform(df['Grupo'])

df_train, df_val = train_test_split(df, test_size=0.25, stratify=df['labels'], random_state=42)
train_dataset = Nifti3DDataset(df_train, le)
val_dataset = Nifti3DDataset(df_val, le)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

#Entrenar el modelo
model = VGG3D(num_classes=len(le.classes_)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, min_lr=1e-8, verbose=True)

for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0, 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc_train = 100 * train_correct / len(train_dataset)
    print(f"Epoch {epoch+1} - Loss: {train_loss:.4f}, Train Acc: {acc_train:.2f}%")

    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    acc_val = 100 * val_correct / len(val_dataset)
    print(f"Test Acc: {acc_val:.2f}%")

torch.save(model.state_dict(), "vgg16_3d_alzheimer3.pth")
print("Modelo guardado como vgg16_3d_alzheimer3.pth")
